pytorch官网： https://pytorch.org/docs/stable

PyTorch常用代码段合集：https://mp.weixin.qq.com/s/2bpT93KrqwWfE-Ob0iC8xA

## 1 张量（Tensor）处理


### 1.1 TORCH.GATHER理解

> torch.gather(input, dim, index, *, sparse_grad=False, out=None) → Tensor



具体是如何得到的呢？
https://blog.csdn.net/edogawachia/article/details/80515038

<img src="https://tva1.sinaimg.cn/large/008vxvgGgy1h80hwytl2kj319u0gugnh.jpg" alt="image-20221111002703552" style="zoom:50%;" />

In [6]:
"""
关键是“指定dim=1，也就是横向，dim=0，也就是纵向”
"""
import torch
t = torch.tensor([[1, 2], [3, 4]])
print(t)
torch.gather(t, 1, torch.tensor([[0, 0], [1, 0]]))

tensor([[1, 2],
        [3, 4]])


tensor([[1, 1],
        [4, 3]])